> **Installing dependencies**

> **Installed torch audio soundfile to use audio datasets**

> **Installed jiwer, evaluate  to use wer metric**

In [1]:
!pip install datasets transformers torchaudio soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: 

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [3]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.0 MB/s eta 0:00:0000:0100:01


> **Built a pipeline using transformers.**

> **Used whisper for audio to speech.**

> **Used biomedical-ner-all for named entity recognition**

In [4]:
from datasets import load_dataset, Audio
from transformers import pipeline

# 1. Load dataset
dataset = load_dataset("jarvisx17/Medical-ASR-EN", split="train")

# Decode audio to waveform
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# 1. Load the ASR pipeline
asr = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

ner = pipeline("ner",
               model="d4data/biomedical-ner-all",
               aggregation_strategy="simple")


# 3. Take one sample (replace with your dataset loading logic)
sample = dataset[0]
audio = sample["audio"]["array"]
ground_truth = sample["transcription"]

# 4. Run ASR
prediction = asr(audio)
transcribed_text = prediction["text"]

print(f"🔊 Ground Truth: {ground_truth}")
print(f"📝 Transcribed Text: {transcribed_text}")

# 5. Run NER on transcribed text
entities = ner(transcribed_text)

# Print the raw entities for debugging
print("\n🏷️ Named Entities:")
for ent in entities:
    print(f" - {ent['word']} ({ent['entity_group']}, score={ent['score']:.2f})")

# 6. Build a medical fact sheet with CORRECT MAPPING
fact_sheet = {
    "Disease": [],
    "Symptom": [],
    "Medication": [],
    "Test": [],
    "Anatomy": []
}

# Define a mapping from the model's labels to our fact sheet categories
label_mapping = {
    'Disease_disorder': "Disease",
    'Sign_symptom': "Symptom",
    'Drug': "Medication",
    'Lab_value': "Test",
    'Biological_structure': "Anatomy", # This is the key fix
    'Body_structure': "Anatomy",       # Some models might use this
    'Body_part': "Anatomy",            # or this
    'Therapeutic_procedure': "Test",
    'Diagnostic_procedure': "Test",
}

for ent in entities:
    original_label = ent["entity_group"]
    value = ent["word"]

    # Find the best category for this label, default to None
    category = label_mapping.get(original_label)

    # Only add it if we found a valid category and it's not already in the list
    if category and value not in fact_sheet[category]:
        fact_sheet[category].append(value)

print("\n📌 Extracted Medical Information:")
print(fact_sheet)

2025-09-05 07:18:37.120864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757056717.308957      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757056717.363039      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/262 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

data/train-00000-of-00013-fd3c831bb1117e(…):   0%|          | 0.00/339M [00:00<?, ?B/s]

data/train-00001-of-00013-e247e67b9a3043(…):   0%|          | 0.00/382M [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


🔊 Ground Truth: I have a painful cramp in my feet
📝 Transcribed Text:  I have a painful cramp in my feet.

🏷️ Named Entities:
 - painful (Detailed_description, score=1.00)
 - cramp (Sign_symptom, score=0.98)

📌 Extracted Medical Information:
{'Disease': [], 'Symptom': ['cramp'], 'Medication': [], 'Test': [], 'Anatomy': []}


**Evaluation of ASR and NER done below.**

In [5]:
import evaluate

# Load WER metric
wer_metric = evaluate.load("wer")

# Example: loop through dataset and compute WER
wer_scores = []
for i in range (5):   # change [:5] to process first 5 samples, or remove for all
    sample=dataset[i]
    audio = sample["audio"]["array"]
    ground_truth = sample["transcription"]

    # 1. Run ASR
    prediction = asr(audio)
    transcribed_text = prediction["text"]

    wer = wer_metric.compute(predictions=[transcribed_text], references=[ground_truth])
    wer_scores.append(wer)

    print(f"\n===== Sample {i+1} =====")
    print(f"🔊 Ground Truth: {ground_truth}")
    print(f"📝 Transcribed Text: {transcribed_text}")
    print(f"📉 WER: {wer:.3f}")
    # 2. Run NER
    entities = ner(transcribed_text)

    print("\n🏷️ Named Entities:")
    for ent in entities:
        print(f" - {ent['word']} ({ent['entity_group']}, score={ent['score']:.2f})")

    # 3. Initialize fact sheet
    fact_sheet = {
        "Disease": [],
        "Symptom": [],
        "Medication": [],
        "Test": [],
        "Anatomy": []
    }

    # 4. Define label mapping
    label_mapping = {
        'Disease_disorder': "Disease",
        'Sign_symptom': "Symptom",
        'Drug': "Medication",
        'Lab_value': "Test",
        'Biological_structure': "Anatomy",
        'Body_structure': "Anatomy",
        'Body_part': "Anatomy",
        'Therapeutic_procedure': "Test",
        'Diagnostic_procedure': "Test",
    }

    # 5. Fill fact sheet
    for ent in entities:
        original_label = ent["entity_group"]
        value = ent["word"]

        category = label_mapping.get(original_label)

        if category and value not in fact_sheet[category]:
            fact_sheet[category].append(value)

    # 6. Print structured info
    print("\n📌 Extracted Medical Information:")
    print(fact_sheet)



===== Sample 1 =====
🔊 Ground Truth: I have a painful cramp in my feet
📝 Transcribed Text:  I have a painful cramp in my feet.
📉 WER: 0.125

🏷️ Named Entities:
 - painful (Detailed_description, score=1.00)
 - cramp (Sign_symptom, score=0.98)

📌 Extracted Medical Information:
{'Disease': [], 'Symptom': ['cramp'], 'Medication': [], 'Test': [], 'Anatomy': []}

===== Sample 2 =====
🔊 Ground Truth: The pain feels like it's right below the skin
📝 Transcribed Text:  The pain feels like it's right below the skin.
📉 WER: 0.111

🏷️ Named Entities:
 - pain (Coreference, score=0.95)
 - right (Biological_structure, score=0.53)
 - below the (Biological_structure, score=0.71)

📌 Extracted Medical Information:
{'Disease': [], 'Symptom': [], 'Medication': [], 'Test': [], 'Anatomy': ['right', 'below the']}

===== Sample 3 =====
🔊 Ground Truth: I feel suicidal.
📝 Transcribed Text:  I feel suicidal.
📉 WER: 0.000

🏷️ Named Entities:

📌 Extracted Medical Information:
{'Disease': [], 'Symptom': [], 'Medicat